<a href="https://colab.research.google.com/github/kotech1/computervision/blob/master/10_neural_style_transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Build a VGG19 model loaded with pre-trained ImageNet weights
model = vgg19.VGG19(weights="imagenet", include_top=False)
model.summary()

In [ ]:
# 추가 손실 함수
# 생성된 이미지에서 base 이미지의 "content"를
# 유지하기 위함

def content_loss(base, combination):
    return tf.reduce_sum(tf.square(combination - base))

In [ ]:
# VGG19의 각 레이어의 특징(feature)를 dict형식으로 반환
feature_extractor = keras.Model(inputs=model.inputs, outputs=outputs_dict)

In [ ]:
# content loss에 사용할
content_layer_name = "block5_conv2"


def compute_loss(combination_image, base_image, style_reference_image):
    input_tensor = tf.concat(
        [base_image, style_reference_image, combination_image], axis=0
    )
    features = feature_extractor(input_tensor)

    # loss 초기화
    loss = tf.zeros(shape=())

    # Add content loss
    layer_features = features[content_layer_name]
    base_image_features = layer_features[0, :, :, :]
    combination_features = layer_features[2, :, :, :]
    loss = loss + content_weight * content_loss(
        base_image_features, combination_features
    )

In [ ]:
# 이미지 텐서의 gram matrix (특징별 outer product)

def gram_matrix(x):
    # 채널을 맨 앞으로, w,h를 뒤로 보냄
    x = tf.transpose(x, (2, 0, 1)) 
    # (c, w, h) ==> (c, w*h)
    features = tf.reshape(x, (tf.shape(x)[0], -1)) 
    # (c, w*h) x (w*h,c) = (c,c) 
    gram = tf.matmul(features, tf.transpose(features))  
    return gram